In [ ]:
from pycalphad import Database, equilibrium, variables as v, calculate
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)

In [ ]:
import random

random.seed(10)
all_components = list(db.elements)
all_phases =  list(db.phases)
all_phases.sort()
all_components.sort()
print('Количество фаз =' + str(len(all_phases)))
print('Количество компонент =' + str(len(all_components)))
print(all_components)
# components = random.sample(all_components, 8)
components = ['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']

print(components)


In [ ]:
counter = 0 
components_phases = {}
for comp in components:
    components_phases[comp] = set()
for item in db.phases.items():
    # item[0] - name, item[1] - content
    # item[1] - name, constituents, sublattices, model_hints
    if len(item[1].constituents)>4:
        # print(item[1].constituents)
        counter += 1
    
    for species in item[1].constituents[0]:
        # name, constituents, charge
        for phase in components_phases.keys():
            if phase in species.name:
                components_phases[phase].add(item[0])
    #     print(species)
    # print(item[1].constituents[0])

print(counter)
print(components_phases)

for comp, phases in components_phases.items():
    mask = [1 if phase in phases else 0 for phase in all_phases]
    components_phases[comp] = int(''.join(map(str, mask)), 2)

In [ ]:
for comp, phases in components_phases.items():
    print(f'{comp}: {bin(phases)}')

In [ ]:
def getPhases(intMask):
    return [all_phases[i] for i in range(len(all_phases)) if (intMask & (1 << i)) > 0]
print(getPhases(components_phases['SE']))

In [ ]:
for comp, phases in components_phases.items():
    print(comp)
    print(getPhases(phases))
    for phase in getPhases(phases):
        if str(phase) in db.symbols:
            print(f'{phase} in {db.symbols}')
print('GSM4SB3' in db.symbols)

In [ ]:
import re

# read the text file
text = ""
function_start = False
with open(db_name, 'r') as f:
    for line in f:
        if "Standard functions for the elements" in line:
            function_start = True
        if "Data for the elements from unary database" in line:
            function_start = False

        if function_start:
            text += line

component_functions = {}
# find all the component names in the text
component_names = re.findall(r'\$ ([A-Z][a-z]*)', text)
for component in component_names:
    # find the text between the component name and the next component name
    pattern = r'\$ {}.*?\$ [A-Z][a-z]*'.format(component)
    component_text = re.search(pattern, text, re.DOTALL).group()
    
    # find all the function names in the component text
    function_names = re.findall(r'FUNCTION ([A-Z]*)', component_text)
    
    component_functions[component] = function_names

In [ ]:
# print the functions for each component
for component, functions in component_functions.items():
    if len(functions) > 0:
        print(component + ':')
        print(functions)